In [ ]:
# 1. NPP_sp
cfg_var_NPP_sp.ODA_path_load(cfg_var_NPP_sp.var)

ODA_ds = xr.open_mfdataset(cfg_var_NPP_sp.ODA_file_list[0][0:10],
                           chunks={'time': 12}, 
                           combine='nested', 
                           concat_dim=[[*cfg_var_NPP_sp.ODA_ensembles[0:10]], 'time'], 
                           parallel=True,
                           preprocess=lambda ds: process_coords(ds, start_date, end_date),
                           decode_cf=True, 
                           decode_times=True)

ODA_ds2 = xr.open_mfdataset(cfg_var_NPP_sp.ODA_file_list[0][10:20], 
                           chunks={'time': 12}, 
                           combine='nested', 
                           concat_dim=[[*cfg_var_NPP_sp.ODA_ensembles[10:20]], 'time'], 
                           parallel=True,
                           preprocess=lambda ds: process_coords(ds, start_date, end_date),
                           decode_cf=True, 
                           decode_times=True)

# en4.2 -> until 2021; projdv7.3 -> until 2020; need to be read separately
ODA_ds_a = []
ODA_ds_a.append(ODA_ds)
ODA_ds_xr = xr.concat(ODA_ds_a, dim='time')
ODA_ds_xr = ODA_ds_xr.expand_dims({'ens': range(10)})

ODA_ds2_a = []
ODA_ds2_a.append(ODA_ds2)
ODA_ds2_xr = xr.concat(ODA_ds2_a, dim='time')
ODA_ds2_xr = ODA_ds2_xr.expand_dims({'ens': range(10, 20)})

cfg_var_NPP_sp.ODA_ds = []
cfg_var_NPP_sp.ODA_ds.append(ODA_ds_xr)
cfg_var_NPP_sp.ODA_ds.append(ODA_ds2_xr)
cfg_var_NPP_sp.ODA_ds = xr.concat(cfg_var_NPP_sp.ODA_ds, dim='ens')
cfg_var_NPP_sp.ODA_ds = cfg_var_NPP_sp.ODA_ds.reindex(ens=list(range(1, 20+1)))
cfg_var_NPP_sp.ODA_ds = cfg_var_NPP_sp.ODA_ds.sortby('time')

cfg_var_NPP_sp.ODA_ds = cfg_var_NPP_sp.ODA_ds.rename({"concat_dim": "ens_ODA"})
new_time = cfg_var_NPP_sp.ODA_ds.time - np.array([datetime.timedelta(days=15)] * len(cfg_var_NPP_sp.ODA_ds.time))
cfg_var_NPP_sp.ODA_ds = cfg_var_NPP_sp.ODA_ds.assign_coords(time=new_time)
cfg_var_NPP_sp.ODA_ds = cfg_var_NPP_sp.ODA_ds.drop('ens')
cfg_var_NPP_sp.ODA_ds=cfg_var_NPP_sp.ODA_ds.mean(dim='ens')